In [2]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import ollama
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [21]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'codellama'
MODEL_ANTHROPIC = 'claude-3-7-sonnet-latest'

In [4]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")


# Connect to OpenAI, Anthropic and ollama

openai = OpenAI()

claude = anthropic.Anthropic()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
API key looks good so far


In [5]:
system_message = "You are an assistant that provides useful comments and docstring for Python code "
system_message += "Respond only with python code plus the comment; do not over comment"
system_message += "This needs to help more users understand what is happening in my code"

In [6]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code with comments to let other users know what is happening. "
    user_prompt += "Respond only with python code; Do not provide any other words than the code and the comments in code. \n\n "

    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [17]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(
        model=MODEL_GPT, 
        messages=messages_for(python), 
        stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    # write_output(reply)

In [18]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=MODEL_ANTHROPIC,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    # write_output(reply)

In [22]:
def optimize_llama(python):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA, # model is defined above
        messages=messages_for(python),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
optimize_llama(pi)

```
import time



In [20]:
optimize_gpt(pi)

```python
import time  # Import the time module to measure execution time

def calculate(iterations, param1, param2):
    result = 1.0  # Initialize the result variable
    # Loop through a range of iterations
    for i in range(1, iterations + 1):
        j = i * param1 - param2  # Calculate j based on the current iteration
        result -= (1 / j)  # Update result by subtracting the inverse of j
        j = i * param1 + param2  # Calculate j with a different operation
        result += (1 / j)  # Update result by adding the inverse of the new j
    return result  # Return the final computed result

start_time = time.time()  # Record the start time
result = calculate(100_000_000, 4, 1) * 4  # Call the calculate function with large iterations and scale the result
end_time = time.time()  # Record the end time

# Print the formatted result and the total execution time
print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
```